# Neural Network Training on Full Dataset

---

This notebook trains a neural network on the whole hourly crypto training data set from `gs://cryptocrystalball_public/CryptoDataset/Hourly/`. Please note that direct access to the Google Bucket folder containing the tick data is not possible, but the individual files are copy to the notebooks runtime a few cells below.

# Todo: Add validfiles file to DS

The notebook is based on the example version for training under [NeuralNetworkTrainingExample.ipynb](
https://github.com/girsigit/CryptoCrystalBall/blob/main/JupyterDocker/notebooks/NeuralNetworkTrainingExample.ipynb).

For a complete run, the notebook takes around 3 hours on the Colab GPU. This includes training for one epoch and the prediction of direction information on a 'unknown' tick file. (Unknown --> The network has never seen it before)

The Git Repo to this notebook, containing more information, can be found under: [The link to the repo](https://github.com/girsigit/CryptoCrystalBall)

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
DEBUG_CONDA_PREFIX = "/home/bernhard/Documents/git/CryptoCrystalBall/conda"
#DEBUG_CONDA_PREFIX = ""

In [ ]:
# Define the checkpoint path for saving model checkpoints
CHECKPOINT_PATH = DEBUG_CONDA_PREFIX + "/checkpoints"

# # If you want to save the checkpoints to a Google Bucket, you have to login for the storage in the Colab session
# from google.colab import auth
# from google.colab._message import MessageError

# try:  
#   auth.authenticate_user()
# except MessageError as me:
#   CHECKPOINT_PATH = "/content"

#   logging.error(me)
#   logging.warning(f"Could not authenticate to Bucket, saving the checkpoints locally to '{CHECKPOINT_PATH}'") 

In [ ]:
# Imports
import sys
import os
from os import listdir
from os.path import isfile, join
import time

In [ ]:
# Check if the notebook is run in google colab, if so, import some stuff
colab = 'google.colab' in sys.modules

if colab:
    print("Running in Colab")

    # Clone the whole repo to get all data and code if not already done
    if not os.path.exists("/content/CryptoCrystalBall"):
      !git clone https://github.com/girsigit/CryptoCrystalBall

      # cd into the notebooks directory --> Necessary to match all paths for importing
    %cd /content/CryptoCrystalBall/JupyterDocker/notebooks

In [ ]:
# Install a plugin for monitoring preformance profiles
!pip install -U tensorboard-plugin-profile

---
# Download the dataset from Google Bucket

It consists of three folders:
- **Train**: Data used for training the network
- **Eval**: Data used for monitoring the training process (check if the network converges)
- **Test**: Used to check the networks predictions on data it has never seen before --> Predict the 'unknown future'


In [ ]:
# Check if the dataset already has been copy, if not, copy it
if not os.path.exists("/content/dataset"):
  !mkdir /content/dataset
  !gsutil -m cp -r gs://cryptocrystalball_public/CryptoDataset/Hourly/* /content/dataset/

In [ ]:
#@markdown ### Use only significant currencies
#@markdown Load a manually defined list of significant currencies (`significant_currencies.txt`).
#@markdown This list contains no currencies which little or no volume or price movement, to
#@markdown avoid training on data sample which would never be used to trade on in a real 
#@markdown application.

#@markdown If enabled, only currency pairs with the base currency USDT are laoded,
#@markdown this is important to prevent interference between different cryptocurrencies.
#@markdown For example, in `BTC-ETH.csv`, there is influence of both the BTC and the ETH price, but we want to predict trade signals based on a 'real' currency (USDT is kind of the same as USD).

significant_only = True #@param {type:"boolean"}

if significant_only:
  with open(DEBUG_CONDA_PREFIX + "/content/dataset/significant_currencies.txt") as f:
    SIGNIFICANT_CURRS = f.read().splitlines()

  print(f"Loaded {len(SIGNIFICANT_CURRS)} significant files names.")

In [ ]:
# Get train file names - Only pick the ones ending with -USDT to prevent
# influence between different currencies
TRAIN_PATH = DEBUG_CONDA_PREFIX + "/content/dataset/Train"

# Get all file names
TRAIN_FILES = [os.path.join(TRAIN_PATH,f) for f in listdir(TRAIN_PATH) if isfile(join(TRAIN_PATH, f)) and ".csv" in f ]

# Filter for significant currencies only
if significant_only:
  TRAIN_FILES = [f for f in TRAIN_FILES if f.split("/")[-1].replace(".csv","") in SIGNIFICANT_CURRS]

# Filter for USDT-based ones only
TRAIN_FILES = [f for f in TRAIN_FILES if "-USDT" in f]

# Sort them (as a stable basis for randomizing afterwards)
TRAIN_FILES = sorted(TRAIN_FILES)

print(f"The train dataset contains {len(TRAIN_FILES)} files.")
print(TRAIN_FILES[:3])

In [ ]:
# Get eval file names - Only pick the ones ending with -USDT to prevent
# influence between different currencies
EVAL_PATH = DEBUG_CONDA_PREFIX + "/content/dataset/Eval"

# Get all file names
EVAL_FILES = [os.path.join(EVAL_PATH,f) for f in listdir(EVAL_PATH) if isfile(join(EVAL_PATH, f)) and ".csv" in f ]

# Filter for significant currencies only
if significant_only:
  EVAL_FILES = [f for f in EVAL_FILES if f.split("/")[-1].replace(".csv","") in SIGNIFICANT_CURRS]

# Filter for USDT-based ones only
EVAL_FILES = [f for f in EVAL_FILES if "-USDT" in f]

# Sort them (as a stable basis for randomizing afterwards)
EVAL_FILES = sorted(EVAL_FILES)

print(f"The evaluation dataset contains {len(EVAL_FILES)} files.")
print(EVAL_FILES[:3])

In [ ]:
# Get train file names - Only pick the ones ending with -USDT to prevent
# influence between different currencies
TEST_PATH = DEBUG_CONDA_PREFIX + "/content/dataset/Test"

# Get all file names
TEST_FILES = [os.path.join(TEST_PATH,f) for f in listdir(TEST_PATH) if isfile(join(TEST_PATH, f)) and ".csv" in f ]

# Filter for significant currencies only
if significant_only:
  TEST_FILES = [f for f in TEST_FILES if f.split("/")[-1].replace(".csv","") in SIGNIFICANT_CURRS]

# Filter for USDT-based ones only
TEST_FILES = [f for f in TEST_FILES if "-USDT" in f]

# Sort them (as a stable basis for randomizing afterwards)
TEST_FILES = sorted(TEST_FILES)

print(f"The test dataset contains {len(TEST_FILES)} files.")
print(TEST_FILES[:3])

---
# Import the libraries

In [ ]:
# Try importing the Ta-Lib library, if this fails, try to install it and
# import it again afterwards
try:
    import talib
except:
    !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
    !tar -xzvf ta-lib-0.4.0-src.tar.gz
    %cd ta-lib
    !./configure --prefix=/usr
    !make
    !make install
    !pip install Ta-Lib
    %cd ..

    import talib

In [ ]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm
import datetime
import pickle
import json

---
# Add custom import path for DataStreamCreator and IndicatorCalculator

These libs are not in the standard python directory, so their paths have to be added to the import paths

In [ ]:
# Get the current directory
current_dir = os.getcwd()
current_dir_splitted = current_dir.split(os.sep)
current_dir_splitted

In [ ]:
# # Create the import directories for the DataStreamCreator and the IndicatorCalculator
# dsc_dir = os.path.join(os.sep, *current_dir_splitted[:-2], 'DataStreamCreator')
# print(f"dsc_dir: {dsc_dir}")

# ind_dir = os.path.join(os.sep, *current_dir_splitted[:-2], 'IndicatorCalculator')
# print(f"ind_dir: {ind_dir}")

# # Add them to the import paths
# sys.path.insert(0, dsc_dir)
# sys.path.insert(0, ind_dir)

In [ ]:
# Create the import directories for the DataStreamCreator and the IndicatorCalculator
dsc_dir = os.path.join(os.sep, *current_dir_splitted[:-2], 'DataStreamCreator')
print(f"dsc_dir: {dsc_dir}")

ind_dir = os.path.join(os.sep, *current_dir_splitted[:-2], 'IndicatorCalculator')
print(f"ind_dir: {ind_dir}")

dsc_dir = "/home/bernhard/Documents/git/CryptoCrystalBall/DataStreamCreator"
ind_dir = "/home/bernhard/Documents/git/CryptoCrystalBall/IndicatorCalculator"

# Add them to the import paths
sys.path.insert(0, dsc_dir)
sys.path.insert(0, ind_dir)

In [ ]:
# Import the actual classes
from IndicatorCalculator import IndicatorCalculator
import DataStreamCreator

---
# Define all the parameters and variables

In [ ]:
# Define the tick data path
DATA_PATH = os.path.join(os.sep, *current_dir_splitted[:-2], 'DemoData')
print(f"DATA_PATH: {DATA_PATH}")

In [ ]:
# Define the chart image save path
IMG_SAVE_PATH = os.path.join(os.sep, *current_dir_splitted[:-2], 'Documentation', 'Images')
print(f"IMG_SAVE_PATH: {IMG_SAVE_PATH}")

In [ ]:
# Define a global random seed
RANDOM_SEED = 11

# Set the seed in np
np.random.seed(RANDOM_SEED)

In [ ]:
# X_BLOCK_LENGHT defines how far into the past a 'slice of a chart' shall be
# See: https://github.com/girsigit/CryptoCrystalBall/tree/main/DataStreamCreator#xblockgenerator
X_BLOCK_LENGHT = 512

In [ ]:
# How many examples shall be processed at the same time, limited by GPU memory
BATCH_SIZE = 256

In [ ]:
# Finanical indicator timespans
# See: https://github.com/girsigit/CryptoCrystalBall/tree/main/IndicatorCalculator
SHORTSPAN = 6
MIDSPAN = 48
LONGSPAN = 120

In [ ]:
# Define how many currencies shall be used in parallel to generate the train data
PARALLEL_GENERATORS = 32

---
# Prepare data source

For training a neural network, first the data source has to be prepared. For this purpose, the method `FileListToDataStream` from the `DataStreamCreator` class is used. This method creates a stream of `X-Block` and `y-data` arrays out of a list of .csv file names, pointing to tick tables (called `EXAMPLE_FILE_PATHS` in this example). For details about `X-Blocks` and `y-data`, please refer to the documentation of the `XBlockGenerator` and the `YDataGenerator` under https://github.com/girsigit/CryptoCrystalBall/tree/main/DataStreamCreator.

<br>

It is important to mention that the last timestamp of an X-Block (a frame from the past) matches the timestamp of the y-data entry (lookout into the future of the price).

<br>

As a target value (y data type), the future direction and its derviation of the price have been chosen, as they are easy to visualize and understand.




In [25]:
# Import the required TensorFlow modules
# Todo: Remove unused ones

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, ReLU, Add, Flatten, Concatenate, LayerNormalization, UpSampling2D, Activation, LSTM, Multiply, Dropout, Reshape, Permute, BatchNormalization, MaxPooling1D, AveragePooling1D, MaxPooling3D, AveragePooling2D, LayerNormalization, MaxPooling2D, UnitNormalization, UpSampling1D
from keras.layers import Conv1D, Conv2D
from keras.models import Model
from keras.optimizers import Adam

from tensorflow.python.lib.io import file_io

2022-12-30 21:24:26.573820: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-30 21:24:27.270113: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-30 21:24:27.270186: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-30 21:24:27.270193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [ ]:
# Set direction and derivation information as y target
# Both y values (direction & derivation) are in the interval [-1.0,1.0]

Y_TYPE_DICT = copy.deepcopy(DataStreamCreator.YDataGenerator.PARAM_DICT_TEMPLATE_Y_DATA_TYPE_DIRECTION_FLOAT)
Y_TYPE_DICT

{'dataType': 0,
 'direction_ma_timespan': 200,
 'derivation_ma_timespan': 100,
 'direction_derivation_shift_span': 0}

---
# Get the feature dimension of the dataset for creating the NN

This is required for setting up the neural network. Maybe this could be done at runtime, but is not necessary, as the indicator configuration, which leads to the feature dimension, has to be the same in the predication phase afterwards and therefore has to be known.

For this purpose, only the first train file is used.

In [ ]:
# Initialize the FileListToDataStream generator
dataStreamTraining = DataStreamCreator.FileListToDataStream(fileList = [TRAIN_FILES[0]],
                                                     batch_size = BATCH_SIZE,
                                                     X_Block_lenght = X_BLOCK_LENGHT,
                                                     y_type_dict=Y_TYPE_DICT,
                                                     parallel_generators=1
                                                    )
# Get the first batch
batch = next(dataStreamTraining)
XShape = batch['X'].shape
yShape = batch['y'].shape
FEATURES = XShape[-1]

print(f"Shape of X-Block array: (BATCH_SIZE, X_BLOCK_LENGHT, FEATURES) -> {XShape}")
print(f"Shape of y-data array: (BATCH_SIZE, DIRECTION & DERVIATION) -> {yShape}")
print(f"FEATURES: {FEATURES}")

INFO:root:File '1INCH-USDT.csv' loaded, 0 left


Shape of X-Block array: (BATCH_SIZE, X_BLOCK_LENGHT, FEATURES) -> (256, 512, 226)
Shape of y-data array: (BATCH_SIZE, DIRECTION & DERVIATION) -> (256, 2)
FEATURES: 226


---
# Create the actual data stream for training

Re-Create the data stream used for finding the feature count to start from a fresh generator.

Please note that the `shuffle` flag is set to `true` here. This provides shuffing of both the input file list (irrelevant in this example) and position where the `X-Blocks` and `y-data` informations are collected.

<br>

## Attention, weird behavior!
A python generator function has to be applied on the dataStream, as the return provided by `FileListToDataStream` method is not a 'real' generator and therefore cannot be used by TF.
```
Todo: Bugfix --> Include this into the FileListToDataStream
```

# Todo pass gen arguments:
https://www.tensorflow.org/guide/data

Consuming Python generators
Another common data source that can easily be ingested as a tf.data.Dataset is the python generator.

The constructor takes a callable as input, not an iterator. This allows it to restart the generator when it reaches the end. It takes an optional args argument, which is passed as the callable's arguments.

In [ ]:
# A python generator function has to be applied on the dataStream

def pythonGeneratorTraining():
  # Initialize the FileListToDataStream generator
  dataStreamTraining = DataStreamCreator.FileListToDataStream(fileList = TRAIN_FILES,
                                                      batch_size = BATCH_SIZE,
                                                      X_Block_lenght = X_BLOCK_LENGHT,
                                                      y_type_dict=Y_TYPE_DICT,
                                                      shuffle=True,
                                                      parallel_generators=PARALLEL_GENERATORS
                                                      )
  
  # This while has to integrated into the FileListToDataStream method
  while True:  
    try:
      ne = next(dataStreamTraining)
      _X = ne['X']
      _y = ne['y']
      
      yield (_X, _y)
    except StopIteration as si:
      logging.warning("StopIteration in pythonGenerator")
      logging.warning(si)
      return

In [ ]:
# Create a Tensorflow dataset out of the python generator, which can be fed to the network
tfGenTrainingSource = tf.data.Dataset.from_generator(pythonGeneratorTraining, 
                                               output_types = (tf.float32, tf.float32), # Todo: Automatically determine data type from dataStreamTraining (float, int for categories)
                                               output_shapes=((XShape), (yShape))
                                               )

# Use some interleave and prefetch methods to speed up the dat pipeline
# Todo: This still needs improvement
# https://www.tensorflow.org/guide/data_performance

tfGenTraining = tf.data.Dataset.range(1).interleave(lambda _: tfGenTrainingSource.prefetch(buffer_size=8),num_parallel_calls=1, cycle_length=2, block_length=4)
tfGenTraining

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2022-12-30 21:24:28.444797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-30 21:24:28.502961: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-30 21:24:28.503129: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-30 21:24:28.503664: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

<ParallelInterleaveDataset element_spec=(TensorSpec(shape=(256, 512, 226), dtype=tf.float32, name=None), TensorSpec(shape=(256, 2), dtype=tf.float32, name=None))>

---
# Create an additional data stream for evaluation

To evaluate the training process, an metrics are calulated on an evaluation data set.

To speed this up, only few files from the training set are used.

In [ ]:
np.random.seed(110)
random_indices = np.random.randint(low=0, high=len(EVAL_FILES)-1, size=4)
np.random.seed(RANDOM_SEED)

EVAL_FILES_CROPPED = list(np.array(EVAL_FILES)[random_indices])
EVAL_FILES_CROPPED

['/home/bernhard/Documents/git/CryptoCrystalBall/conda/content/dataset/Eval/1INCH-USDT.csv',
 '/home/bernhard/Documents/git/CryptoCrystalBall/conda/content/dataset/Eval/TFC-USDT.csv',
 '/home/bernhard/Documents/git/CryptoCrystalBall/conda/content/dataset/Eval/KLV-USDT.csv',
 '/home/bernhard/Documents/git/CryptoCrystalBall/conda/content/dataset/Eval/BTCV-USDT.csv']

In [ ]:
# A python generator function has to be applied on the dataStream

def pythonGeneratorEval():
  # Initialize the FileListToDataStream generator
  dataStreamEval = DataStreamCreator.FileListToDataStream(fileList = EVAL_FILES_CROPPED,
                                                      batch_size = BATCH_SIZE,
                                                      X_Block_lenght = X_BLOCK_LENGHT,
                                                      y_type_dict=Y_TYPE_DICT,
                                                      shuffle=True,
                                                      parallel_generators=1
                                                      )
  
  # This while has to integrated into the FileListToDataStream method
  while True:  
    try:
      ne = next(dataStreamEval)
      _X = ne['X']
      _y = ne['y']
      
      yield (_X, _y)
    except StopIteration as si:
      logging.warning("StopIteration in pythonGenerator")
      logging.warning(si)
      return

In [ ]:
# Create a Tensorflow dataset out of the python generator, which can be fed to the network
tfGenEval = tf.data.Dataset.from_generator(pythonGeneratorEval, 
                                               output_types = (tf.float32, tf.float32), # Todo: Automatically determine data type from dataStreamTraining (float, int for categories)
                                               output_shapes=((XShape), (yShape))
                                               )
tfGenEval = tfGenEval.prefetch(buffer_size=2)
tfGenEval

<PrefetchDataset element_spec=(TensorSpec(shape=(256, 512, 226), dtype=tf.float32, name=None), TensorSpec(shape=(256, 2), dtype=tf.float32, name=None))>

---
# Create the neural network

In the example, two Conv1D pyramids are used to extract features at different levels out of the feature and the time dimension.

It is also possible to use different architecures, like LSTM, Conv2D, Attention-Based ones or even NLP-based models. I have already tried many different ones and will publish them into the model zoo.

```
Todo: Create Model Zoo
```

```
Todo: Create Image of netwrok structure
```

In [ ]:
# Create the FPNWithFScaledInput model
# Infos about the feature pyramid: https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c

def CreateModelFPNWithFScaledInput():
  # Define model parameters
  mp = {
      "NAME": "FPNWithFScaledInput",
      "VERSION": 1,
      "CNN_INITIAL_FILTERS": 32, 
      "CNN_EXTRACTOR_LAYERS": 6,
      "CNN_INITIAL_KERNEL_SIZE": 2,
      "CNN_MAX_KERNEL_SIZE": 8,
      "FEATURE_PYRAMID_START_INDEX": 2,
      "HEAD_LSTM_SIZE": 32,
      "HEAD_DENSE_SIZE": 8
      }
  
  # Create the model input  
  inputTicksAndIndicators = Input(shape=(X_BLOCK_LENGHT, FEATURES), name='inputTicksAndIndicators', dtype='float32')

  # Normalize the input data, as it has a wide value spread (range of 1e8)
  normalized = UnitNormalization(name="UnitNormalizationInput", axis=-2)(inputTicksAndIndicators)

  # ----------------------------------------------------------------------------

  # As an example a 1D-CNN pyramid is used to extract higher-level features out
  # of the indicator+tick dimension
  conv_layers = []
  aligned_layers = []
  feature_map_layers = []
  anti_aliased_layers = []
  anti_aliased_pyramid_indices = []
  regressor_layers = []

  finalFilters = mp["CNN_INITIAL_FILTERS"] * 2**(mp["FEATURE_PYRAMID_START_INDEX"])
  print(finalFilters)

  for i in range(mp["CNN_EXTRACTOR_LAYERS"]):
    if 0 == i:
      source_layer = normalized
    else:
      source_layer = feature_conv
    
    filters = int(mp["CNN_INITIAL_FILTERS"]*(2**i))
    kernel_size = mp["CNN_INITIAL_KERNEL_SIZE"]*(2**i)
    kernel_size = int(np.min([mp["CNN_MAX_KERNEL_SIZE"], kernel_size]))
    strides = int(np.min([2,kernel_size]))

    print(f"Step {i}, using {filters} filters with a kernel size of {kernel_size} at strides of {strides}")

    feature_conv = Conv1D(filters=filters,
                          kernel_size=kernel_size,
                          strides=strides, 
                          padding="same",
                          name=f"Feature_Conv1D_{i}")(source_layer)
    
    conv_layers.append(feature_conv)

    # Apply the feature extraction pyramid
    if mp["FEATURE_PYRAMID_START_INDEX"] <= i:
      # Apply a 1x1 convolution to align the channel depth    
      if feature_conv.shape[-1] != finalFilters:
        aligned_layer = Conv1D(filters=finalFilters,
                              kernel_size=1,
                              strides=1,
                              padding="same",
                              name=f"Channel_Depth_Alignment_{i}")(feature_conv)
      else:
        aligned_layer = feature_conv
      
      aligned_layers.append(aligned_layer)

  # Go top-down through the aligned_layers to create the feature_map_layers
  for i in range(mp["CNN_EXTRACTOR_LAYERS"]-1, mp["FEATURE_PYRAMID_START_INDEX"]-1, -1):
    aligned_layers_index = i - mp["FEATURE_PYRAMID_START_INDEX"]
    aligned_layer = aligned_layers[aligned_layers_index]

    print(f"Building feature_map_layers, step {i}. Shape of aligned_layer: {aligned_layer.shape}")

    # The highest-filtered layer is taken as feature map directly
    if i == mp["CNN_EXTRACTOR_LAYERS"]-1:
      feature_map_layers.append(aligned_layer)
      anti_aliased_layers.append(feature_map_layers[-1])
    else:
      # Take the last feature_map_layer and scale it by two
      upsampled = UpSampling1D(size=2,
                               name=f"Pyramid_Upsampling_{i}")(feature_map_layers[-1])

      # Add the aligned_layer
      added = Add(name=f"Pyramid_Add_{i}")([upsampled, aligned_layer])

      # Append as new feature_map_layer
      feature_map_layers.append(added)

      # Apply a convolution with a kernel size of 3 to "reduce the aliasing effect"
      anti_aliased_layer = Conv1D(filters=feature_map_layers[-1].shape[-1],
                                  kernel_size=3,
                                  strides=1,
                                  padding="same",
                                  name=f"Anti_Alias_{i}")(feature_map_layers[-1])

      anti_aliased_layers.append(anti_aliased_layer)

    # Helper storage to keep pyramid index consistent, also in predictor/regressor
    anti_aliased_pyramid_indices.append(i)

  # Apply the predictor head to each feature dimension layer
  for n in range(len(anti_aliased_layers)):
    predictor_input = anti_aliased_layers[n]

    # Get the pyramid index
    pyramid_index = anti_aliased_pyramid_indices[n]

    predictor = LSTM(units=mp["HEAD_LSTM_SIZE"],
                     name=f"Predictor_Head_{pyramid_index}_LSTM")(predictor_input)
    
    # Direction and derivation regressor
    regressor = Dense(units=mp["HEAD_DENSE_SIZE"],
                      name=f"Predictor_Head_{pyramid_index}_Dense_Regressor",
                      activation='relu')(predictor)
    
    regressor_layers.append(regressor)
  
  # Add all regressors together
  regressor_added = Add(name="Add_regressors")(regressor_layers)

  # The output are two values (direction and derivation) in the range [-1.0, 1.0]
  output = Dense(2, activation='tanh', name="Output")(regressor_added)
  outputs = [output]

  # Combine it all into a model object
  model = Model(inputs=inputTicksAndIndicators, outputs=outputs, name=mp["NAME"] + "_" + str(mp["VERSION"]))

  return model, mp

model, model_config = CreateModelFPNWithFScaledInput()
print(model_config)
model.summary(line_length=220)

128
Step 0, using 32 filters with a kernel size of 2 at strides of 2
Step 1, using 64 filters with a kernel size of 4 at strides of 2
Step 2, using 128 filters with a kernel size of 8 at strides of 2
Step 3, using 256 filters with a kernel size of 8 at strides of 2
Step 4, using 512 filters with a kernel size of 8 at strides of 2
Step 5, using 1024 filters with a kernel size of 8 at strides of 2
Building feature_map_layers, step 5. Shape of aligned_layer: (None, 8, 128)
Building feature_map_layers, step 4. Shape of aligned_layer: (None, 16, 128)
Building feature_map_layers, step 3. Shape of aligned_layer: (None, 32, 128)
Building feature_map_layers, step 2. Shape of aligned_layer: (None, 64, 128)
{'NAME': 'FPNWithFScaledInput', 'VERSION': 1, 'CNN_INITIAL_FILTERS': 32, 'CNN_EXTRACTOR_LAYERS': 6, 'CNN_INITIAL_KERNEL_SIZE': 2, 'CNN_MAX_KERNEL_SIZE': 8, 'FEATURE_PYRAMID_START_INDEX': 2, 'HEAD_LSTM_SIZE': 32, 'HEAD_DENSE_SIZE': 8}
Model: "FPNWithFScaledInput_1"
_____________________________

In [ ]:
# Create an optimizer
optimizer = Adam(
    learning_rate=1e-05,
    epsilon=1e-05,
    decay=0.01)

In [ ]:
# Compile the model
# To predict the float values of direction and derivation, the 'Mean Squared Error' loss is used
# In addition, the 'Mean Average Error' is calculated

model.compile(
    optimizer = optimizer,
    loss = ['mse'], 
    metrics=['mae'])

In [ ]:
# Load the TensorBoard notebook extension for visualization
%load_ext tensorboard

log_dir = DEBUG_CONDA_PREFIX + "/content/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    update_freq=1,
    profile_batch=(5,100)
    )

2022-12-30 21:24:30.074705: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2022-12-30 21:24:30.074724: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2022-12-30 21:24:30.074761: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1664] Profiler found 1 GPUs
2022-12-30 21:24:30.075137: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2022-12-30 21:24:30.187608: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2022-12-30 21:24:30.187782: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1798] CUPTI activity buffer flushed


---
# Create a class to save checkpoints and preform evaluation

In [ ]:
#@title Checkpoint saving class for training
# Todo: Where does model variable come from???
class CheckpointSaverCallback(tf.keras.callbacks.Callback):
  '''
  The `CheckpointSaverCallback` class is, as its name suggests, used to save model checkpoints during the training process.
  This is important, espescially in Google Colab, as the runtime can be disconnected at every time, before the model
  can be saved manually, so the training process would be lost.

  It saves three files: The model architecture and the trained weights (--> `model.h5`), the weights of the optimizer
  to continue the training process as it stopped (-> `w.pickle`) and the optimizer config (-> `c.pickle`).
  Each checkpoint is in a folder named by the epoch it was stored at. The parent folder of all epochs is named like the model.

  Requried constructor arguments:    
  - `checkpoint_path`: A `string` defining the path of the checkpoint folder. It can be a local file path, but also one in a Google Bucket (like `gs://my_fancy_crypto_bucket/chk/`).
  - `model_name`: A `string` defining the model name. Todo: Make this automatically.
  - `save_freq`: An `int` defining the frequency at which checkpoints shall be saved.

  Optional constructor arguments:    
  - `start_epoch`: An `int` defining the file number of the first epoch. This is used if the training is continued from a saved file, and therefore not started at epoch 0.
  - `val_freq`: An `int` defining the frequency at which calculation of the loss values on the eval dataset shall be done.
  - `val_gen`: A `generator? Todo: Define exactly` providing the eval dataset.

  '''

  def __init__(self, checkpoint_path, model_name, save_freq, start_epoch=0, val_freq=0, val_gen=None):

    self.save_freq = save_freq
    self.checkpoint_path = checkpoint_path
    self.model_name = model_name
    
    self.start_epoch = start_epoch
    self.val_freq = val_freq
    self.val_gen = val_gen

    self.current_epoch = 0

  # Set the current_epoch on its beginning
  def on_epoch_begin(self, epoch, logs=None):
    self.current_epoch = epoch + self.start_epoch

  # Save the model on the end of each epoch
  # Batch is set to 'end'
  # Also calculate the eval metrics
  def on_train_epoch_end(self, logs=None):
    self.saveModel('end')
    self.calcEval(self.val_freq)

  # At the end of every batch, check if saving or evaluation is required
  def on_train_batch_end(self, batch, logs=None):
    # Check for save
    if 0 != batch and 0 == batch % self.save_freq:
      self.saveModel(batch)

    # Calculate eval (batch number checks are inside the method)  
    self.calcEval(batch)

    # Check for eval
  def calcEval(self, batch):
    if self.val_gen is not None and 0 < self.val_freq and 0 < batch and 0 == batch % self.val_freq:
      logging.info("\n-------------------------EVAL-------------------------")
      model.evaluate(self.val_gen)
      logging.info("\n-------------------------EVAL-------------------------")

  def saveModel(self, batch):      
    save_folder = os.path.join(self.checkpoint_path,
                              self.model_name,
                              "cp_epoch{:01d}_batch{:05d}".format(self.current_epoch, batch)
                              )

    # Create the save paths
    # The model is first saved locally, then copied to the destination / bucket
    # Optimizer weights and config can be saved directly
    if not os.path.exists(DEBUG_CONDA_PREFIX + "/content/tmp/"):
      os.mkdir(DEBUG_CONDA_PREFIX + "/content/tmp/")

    model_path_local = os.path.join(DEBUG_CONDA_PREFIX + "/content/tmp/", "model.h5")
    model_path_bucket = os.path.join(save_folder, "model.h5")

    path_splitted = model_path_bucket.split(os.path.sep)[1:]
    path_splitted = path_splitted[:-1]

    for i in range(1,len(path_splitted)+1):
      sub_path = os.path.join("/", *path_splitted[:i])     

      if not os.path.exists(sub_path):
        os.mkdir(sub_path)
        logging.info(f"Created '{sub_path}'")

    # Save model locally
    model.save(model_path_local)
    
    # Copy model.h5 over to Google Cloud Storage
    with file_io.FileIO(model_path_local, mode='rb') as input_f:
      with file_io.FileIO(model_path_bucket, mode='wb+') as output_f:
        output_f.write(input_f.read())
    
    logging.info("\nSaved model to: '" + model_path_bucket + "'")

    # Save the model config
    # The model config (how many Conv layers, layer sizes, ...) is stored in a workaround as dict in model.name
    #model_config = eval(model.name)
    model_config_json = json.dumps(model_config, indent = 2) # Todo: Pass the config as parameter

    fp = os.path.join(save_folder, "model_config.json")
    with file_io.FileIO(fp, mode='wb+') as handle:
      handle.write(model_config_json)
      logging.info(f"Saved model config to: '{fp}'")

    # Save optimizer config
    c = copy.deepcopy(self.model.optimizer.get_config())

    fp = os.path.join(save_folder, "c.pickle")
    with file_io.FileIO(fp, mode='wb+') as handle:
      pickle.dump(c, handle, protocol=pickle.HIGHEST_PROTOCOL)
      logging.info(f"Saved optimizer config to: '{fp}'")

    # Save optimizer weights
    # Todo: Not working!!

    #     <ipython-input-38-23423cc1a59d> in saveModel(self, batch)
    #      96     fp = os.path.join(save_folder, "w.pickle")
    #      97     with file_io.FileIO(fp, mode='wb+') as handle:
    # ---> 98       pickle.dump(w, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #      99       logging.info(f"Saved optimizer weights to: '{fp}'")

    # TypeError: Expected binary or unicode string, got <pickle.PickleBuffer object at 0x7fd6ec39a9c0>


    # w = copy.deepcopy(self.model.optimizer.get_weights())

    # fp = os.path.join(save_folder, "w.pickle")
    # with file_io.FileIO(fp, mode='wb+') as handle:
    #   pickle.dump(w, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #   logging.info(f"Saved optimizer weights to: '{fp}'")

In [ ]:
checkpoint_saver = CheckpointSaverCallback(checkpoint_path = CHECKPOINT_PATH,
                                           model_name = model.name,
                                           save_freq = 1000,
                                           start_epoch = 6,
                                           val_freq = 500,
                                           val_gen = tfGenEval)

---
# Train the network

By default, only one training epoch is performed to limit the time needed to finish training. A problem on running the training in Google Colab is that the maximum runtime of a notebook is limited, so the training will be interrupted after a certain time (I think 8-12 hour in the Pro version of Colab).

One epoch of training takes around **2 1/2 hours** on the GPU, during this time, you also have to keep the tab open in the browser and your computer running.


**Hint:** If you dont want to wait for the training to finish, you can also try the pre-trained version from the model zoo. There is a cell in the XXXXXXXX section to load the model.

```
Todo: Add link to pre-trained model
```

In [ ]:
# Show the metrics in TensorBoard
# Hint: If it says 'no data' just switch the categories in the top bar, then it refreshes and works
#%tensorboard --logdir /home/bernhard/Documents/git/CryptoCrystalBall/conda/content/logs

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
model.load_weights("/home/bernhard/Documents/git/CryptoCrystalBall/conda/cp_epoch5_batch02000.h5")

In [ ]:
#model.optimizer.learning_rate = 1e-6

In [ ]:
# Train the model
model.fit(tfGenTraining,
          epochs=200,
          verbose=1,
          callbacks=[tensorboard_callback, checkpoint_saver]
          )

Epoch 1/200


INFO:root:File 'UBQ-USDT.csv' loaded, 120 left
INFO:root:File 'HDAO-USDT.csv' loaded, 119 left
INFO:root:File 'SMARTCREDIT-USDT.csv' loaded, 118 left
INFO:root:File 'BSV-USDT.csv' loaded, 117 left
INFO:root:File 'VET-USDT.csv' loaded, 116 left
INFO:root:File 'MCH-USDT.csv' loaded, 115 left
INFO:root:File 'KLAY-USDT.csv' loaded, 114 left
INFO:root:File 'CKB-USDT.csv' loaded, 113 left
INFO:root:File 'ZRX-USDT.csv' loaded, 112 left
INFO:root:File 'PANDO-USDT.csv' loaded, 111 left
INFO:root:File 'REV-USDT.csv' loaded, 110 left
INFO:root:File 'MYST-USDT.csv' loaded, 109 left
INFO:root:File 'WAVES-USDT.csv' loaded, 108 left
INFO:root:File 'REVV-USDT.csv' loaded, 107 left
INFO:root:File 'TFC-USDT.csv' loaded, 106 left
INFO:root:File 'UMA-USDT.csv' loaded, 105 left
INFO:root:File 'COMP-USDT.csv' loaded, 104 left
INFO:root:File 'AAVE-USDT.csv' loaded, 103 left
INFO:root:File 'CLT-USDT.csv' loaded, 102 left
INFO:root:File 'HIVE-USDT.csv' loaded, 101 left
INFO:root:File '4ART-USDT.csv' loaded, 10

      4/Unknown - 29s 104ms/step - loss: 0.0944 - mae: 0.1916

2022-12-30 21:24:59.662896: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2022-12-30 21:24:59.662917: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.


      6/Unknown - 30s 165ms/step - loss: 0.0949 - mae: 0.1948WARNING:tensorflow:Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0420s vs `on_train_batch_begin` time: 0.0496s). Check your callbacks.


...

     34/Unknown - 19s 556ms/step - loss: 0.0710 - mae: 0.1632

34/34 [==============================] - 20s 566ms/step - loss: 0.0710 - mae: 0.1632


INFO:root:
-------------------------EVAL-------------------------


2637/3475 [=====================>........] - ETA: 13:18 - loss: 0.0758 - mae: 0.1649

KeyboardInterrupt: ignored

In [ ]:
2000/3475 [================>.............] - ETA: 43:36 - loss: 0.0806 - mae: 0.1721

INFO:root:Created '/checkpoints/FPNWithFScaledInput_1/cp_epoch5_batch02000'
INFO:root:
Saved model to: '/checkpoints/FPNWithFScaledInput_1/cp_epoch5_batch02000/model.h5'
INFO:root:Saved model config to: '/checkpoints/FPNWithFScaledInput_1/cp_epoch5_batch02000/model_config.json'
INFO:root:Saved optimizer config to: '/checkpoints/FPNWithFScaledInput_1/cp_epoch5_batch02000/c.pickle'
INFO:root:
-------------------------EVAL-------------------------
INFO:root:File 'KLV-USDT.csv' loaded, 3 left

      6/Unknown - 7s 962ms/step - loss: 0.0486 - mae: 0.1418

INFO:root:File '1INCH-USDT.csv' loaded, 2 left

     15/Unknown - 16s 1s/step - loss: 0.0917 - mae: 0.1820

INFO:root:File 'BTCV-USDT.csv' loaded, 1 left

     25/Unknown - 26s 1s/step - loss: 0.0764 - mae: 0.1675

INFO:root:File 'TFC-USDT.csv' loaded, 0 left

     34/Unknown - 36s 1s/step - loss: 0.0715 - mae: 0.1648

WARNING:root:StopIteration in pythonGenerator
WARNING:root:

34/34 [==============================] - 37s 1s/step - loss: 0.0715 - mae: 0.1648

INFO:root:
-------------------------EVAL-------------------------

2024/3475 [================>.............] - ETA: 43:04 - loss: 0.0757 - mae: 0.1696

---------------------------------------------------------------------------

KeyboardInterrupt                         Traceback (most recent call last)


In [ ]:
# model.save("/home/bernhard/Documents/git/CryptoCrystalBall/conda/checkpoints/long1.h5")

In [ ]:
# Save the train result
#model.save("/content/ModelExample.h5")

---
# Predict on a file of the test set

The next step is to use the network to predict the price direction on a tick file from the test set, which contains data from a time frame the network has never seen before.

In [ ]:
# Choose one index from the test files list
TEST_FILE_INDEX = 108

print(f"The test file path is: '{TEST_FILES[TEST_FILE_INDEX]}'")

In [ ]:
# Set a seperate batch size for testing and predicting
# Please note that if a batch size more than 1 is chosen, it may occur that the
# 'end' of the dataset is not predicted, as no full batch can be filled any more
# and the input tick data is disposed
BATCH_SIZE_TEST = 32

In [ ]:
# A python generator function has to be applied on the dataStream
def pythonGeneratorTest():
  # Initialize the FileListToDataStream generator
  dataStreamTest = DataStreamCreator.FileListToDataStream(fileList = [TEST_FILES[TEST_FILE_INDEX]],
                                                      batch_size = BATCH_SIZE_TEST,
                                                      X_Block_lenght = X_BLOCK_LENGHT,
                                                      y_type_dict=Y_TYPE_DICT,
                                                      shuffle=False, # For visualization
                                                      parallel_generators=1
                                                      )
  while True:  
    try:
      ne = next(dataStreamTest)
      _X = ne['X']
      _y = ne['y']
      
      yield (_X, _y)
    except StopIteration as si:
      logging.warning("StopIteration in pythonGenerator")
      logging.warning(si)
      return

In [ ]:
# Create a Tensorflow dataset, which can be fed to the network
tfGenTest = tf.data.Dataset.from_generator(pythonGeneratorTest, 
                                               output_types = (tf.float32, tf.float32), # Todo: Automatically from dataStreamTraining
                                               output_shapes=((BATCH_SIZE_TEST,XShape[1],XShape[2]), (BATCH_SIZE_TEST,yShape[1]))
                                               )
tfGenTest

In [ ]:
# Calculate the networks metrics on the test datasets file

metrics_test = model.evaluate(tfGenTest)
mse_test, mae_test = metrics_test
print(f"MSE Test: {mse_test}")
print(f"MAE Test: {mae_test}")

---
# Visualize the predictions

Other to the previous visualization, instead of a batch size of 1 the normal size is used to speed up the process. Doing this, some data in the end of the table is ignored, but that makes no difference here, as it is just for visualization.

In [ ]:
# Predict the direction and derivation - on colab, this takes around 3 1/2 minutes
pTest = model.predict(tfGenTest)
pTest.shape

In [ ]:
# Get the real y values (-> Ground Truth) out of the tfGenTest generator for plotting and comparison

# Create numpy generator
it = tfGenTest.as_numpy_iterator()

# Init the values
yRealTest = None
i = 0

# Get the data
while True:
  try:
    X,y = next(it)

    if yRealTest is None:
      yRealTest = y
    else:
      yRealTest = np.concatenate([yRealTest, y])

    i += 1
    if i % 1000 == 0:
      print(i)

  except StopIteration:
    break

print(yRealTest.shape)

In [ ]:
# Split the predicted data into two values for easier processing
p_test_dir = pTest[:,0]
p_test_dir_derivation = pTest[:,1]

In [ ]:
# Also split the real y values
ground_truth_y_dir = yRealTest[:,0]
ground_truth_y_dir_derivation = yRealTest[:,1]

In [ ]:
# Plot the prediction and the ground truth
chart_name="Predicted direction of price movement - unknown currency"
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.set_title(chart_name, fontsize=14)

_ = ax1.plot(ground_truth_y_dir, color="gray", label="Real y direction (Ground truth)")
_ = ax1.plot(p_test_dir, color="blue", label="Predicted y direction")

_= plt.legend(loc="upper left")

ax1.set_xlabel("Timesteps")
ax1.set_ylabel("Direction")

# Save the figure
# fig_name = "FileListToDataStreamExample_" + chart_name.replace(" ", "_").replace("'", "").replace("(", "").replace(")", "") + ".svg"
# plt.savefig(os.path.join(IMG_SAVE_PATH, fig_name), bbox_inches="tight")

In [ ]:
# Load and crop the according tick data
tickdataTest = pd.read_csv(TEST_FILES[TEST_FILE_INDEX])
tickdataTest.set_index("startsAt", inplace = True) # Todo: Take care of the index name
tickdataTest.sort_index()

tickDataTestResized = tickdataTest.iloc[X_BLOCK_LENGHT:,]
tickDataTestResized

### Todo: Add zero line to charts

In [ ]:
# Plot the prediction again, now over the open price
chart_name="Predicted direction of price movement with open price - unknown currency"
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.set_title(chart_name, fontsize=14)

_ = ax1.plot(p_test_dir, color="blue", label="Predicted y direction")

ax2 = ax1.twinx()

ax2.plot(tickDataTestResized.loc[:,'open'].values, color="black", label="Price")

ax1.set_xlabel("Timesteps")
ax1.set_ylabel("Direction")
ax2.set_ylabel("Price")

# Show a legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc="upper left")

# Save the figure
# fig_name = "FileListToDataStreamExample_" + chart_name.replace(" ", "_").replace("'", "").replace("(", "").replace(")", "") + ".svg"
# plt.savefig(os.path.join(IMG_SAVE_PATH, fig_name), bbox_inches="tight")

In [ ]:
# Plot the derivation
chart_name="Predicted derivation of price movement with open price - unknown currency"
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.set_title(chart_name, fontsize=14)

_ = ax1.plot(p_test_dir_derivation, color="red", label="Predicted y derivation")

ax2 = ax1.twinx()

ax2.plot(tickDataTestResized.loc[:,'open'].values, color="black", label="Price")

ax1.set_xlabel("Timesteps")
ax1.set_ylabel("Derivation")
ax2.set_ylabel("Price")

# Show a legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc="upper left")

# Save the figure
# fig_name = "FileListToDataStreamExample_" + chart_name.replace(" ", "_").replace("'", "").replace("(", "").replace(")", "") + ".svg"
# plt.savefig(os.path.join(IMG_SAVE_PATH, fig_name), bbox_inches="tight")

In [ ]:
# Plot a colored price line slice depending on the predicted direction
SLICE_START_INDEX = 0
SLICE_END_INDEX = 100000

chart_name="Colored price line based on predicted direction - unknown currency"
fig, ax1 = plt.subplots(figsize=(20,10))
ax1.set_title(chart_name, fontsize=14)

priceValues = tickDataTestResized.loc[:,'open'].values[SLICE_START_INDEX:SLICE_END_INDEX]
priceValues = priceValues[:p_test_dir.shape[0]]

# Select the rising price values
priceValuesRising = copy.deepcopy(priceValues)
priceValuesRising[p_test_dir[SLICE_START_INDEX:SLICE_END_INDEX] < 0.0] = np.nan

# Select the falling price values
priceValuesFalling = copy.deepcopy(priceValues)
priceValuesFalling[p_test_dir[SLICE_START_INDEX:SLICE_END_INDEX] > -0.25] = np.nan

ax1.plot(priceValues, color="lightgray", label="Price")
ax1.plot(priceValuesRising, color="green", label="Price rising")
ax1.plot(priceValuesFalling, color="red", label="Price falling")

ax1.set_xlabel("Timesteps")
ax1.set_ylabel("Price")

# Show a legend
plt.legend(loc="upper left")

# Save the figure
# fig_name = "FileListToDataStreamExample_" + chart_name.replace(" ", "_").replace("'", "").replace("(", "").replace(")", "") + ".svg"
# plt.savefig(os.path.join(IMG_SAVE_PATH, fig_name), bbox_inches="tight")

---
# Conclusion

The network trained in this notebook can be used in a live application to predict the current direction of price movement. For this purpose, the tick data is fetched every hour, prepared into one `X-Block`, which then can be fed into the network. The output is the prediction of the direction and its derivation at 'now' time. A possible trade strategy would now be to enter at the 'green' chart region and exit on the 'red' one.

A tool for generating such signals can be found under [SignalGenerator](https://github.com/girsigit/CryptoCrystalBall/tree/main/SignalGenerator), but it is still in development and not yet plug-and-play compatible with this notebook!

Also, a warning if it comes to trading with real money: It is possible that the signals are wrong, so if you use this software for real trading, be careful and think about what you are doing, or you may loose money! In the end, you are responsible for your trade decisions!
